[문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough 
- itemgetter()

세션ID 설정하고, 대화를 진행합니다. (multi-turn conversation) 
LLM 모델이 과거 대화를 알지(기억) 못하는 상황을 만드세요.

In [14]:
## 1. 모듈(파일, 라이브러리) 읽어오기
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI
from langchain_core.messages import trim_messages

from dotenv import load_dotenv


## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()

## 3. 파일 읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()
    law = law[:2000]
## 4. prompt 생성
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)


In [15]:

## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model='gpt-3.5-turbo',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    max_tokens=300,
)

## 6-1. trim 설정 
trimmer = trim_messages(
    max_token = 65,
    strategy = 'last',
    token_counter = llm,
    include_system = True,
    allow_partial = False,
    start_on = 'human',   #HumanMessage부터 자르기 시작 
)


from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

## 6-2. chain 구성
chain = (RunnablePassthrough.assign(chat_history=itemgetter('chat_history'))
    | prompt 
    | llm)

chain


RunnableAssign(mapper={
  chat_history: RunnableLambda(itemgetter('chat_history'))
})
| ChatPromptTemplate(input_variables=['law', 'query'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], ty

In [ ]:

## 7. 세션별 대화 이력(히스토리)을 저장할 임시 메모리 저장소
store = {}

## 8. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


## 10. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

## 11. 실행
while True:
    query = input('질문 입력하세요. 종료하려면 s 입력>> ')
    if query.upper() == 'S':
        break     
    with_message_history.invoke(
        {'law': law, 'query': query},
        config={'configurable': {'session_id': 'ywgw'}}
    )
    print('\n'+ '='*50 + '\n')

In [ ]:
messages_list = get_history('ywgw').messages
messages_list
for m in messages_list:
    print(f'{m.type.upper()}=====',m.content)

[HumanMessage(content='육아휴직 남성도 사용 가능?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네, 남성도 육아휴직을 사용할 수 있습니다.\n\n「남녀고용평등법」 제75조(육아휴직)에 따르면, "여자 근로자가 출산 또는 양육을 위하여 휴직하고자 하는 경우에는 이 법 제74조에 따른 육아휴직을 사용할 수 있다."고 되어 있습니다. 이에 따라 육아휴직은 여성 뿐만 아니라 남성도 사용할 수 있는 제도입니다.\n\n또한, 남성의 육아휴직에 관한 부분은 「고용보험법」, 「육아휴직 등 요건 자구 등에 관한 규칙」에서도 상세하게 규정하고 있으니, 해당 법령을 함께 참고하시면 도움이 될 것입니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run--71e49d71-2491-4afc-9138-5da6c3084f0f-0'),
 HumanMessage(content='육아휴직 기간은?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='육아휴직의 기간은 「고용보험법 시행령」 제21조에 따라서 다음과 같이 정해져 있습니다.\n\n1. 만 6세 이하 자녀 1명을 양육하는 근로자: 1년\n2. 만 6세 이하 자녀 2명을 양육하는 근로자: 추가로 6개월\n3. 만 6세 이하 자녀 3명을 양육하는 근로자: 추가로 6개월\n\n따라서, 만 6세 이하 자녀를 양육하는 근로자는 총 최대 1년 6개월 동안 육아휴직을 사용할 수 있습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run--702bcf0c-13bf-445e-9cc7-5ca

In [ ]:
%whos

Variable                         Type                          Data/Info
------------------------------------------------------------------------
BaseChatMessageHistory           ABCMeta                       <class 'langchain_core.ch<...>.BaseChatMessageHistory'>
ChatOpenAI                       ModelMetaclass                <class 'langchain_openai.<...>_models.base.ChatOpenAI'>
ChatPromptTemplate               ModelMetaclass                <class 'langchain_core.pr<...>chat.ChatPromptTemplate'>
InMemoryChatMessageHistory       ModelMetaclass                <class 'langchain_core.ch<...>emoryChatMessageHistory'>
RunnableWithMessageHistory       ModelMetaclass                <class 'langchain_core.ru<...>nableWithMessageHistory'>
StreamingStdOutCallbackHandler   type                          <class 'langchain_core.ca<...>ngStdOutCallbackHandler'>
chain                            RunnableSequence              first=ChatPromptTemplate(<...>******'), streaming=True)
file                 